In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_async import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Suppress Errors
util.logToFile(DATAPATH.parent.parent / 'log' / 'ztest.log')

# Making `Rule of 25` for margins
- [x] Get undPrices with IVs
- [x] Get / Make the option chains
- [ ] Compute margins from unds
- [ ] Check margins against sample options
- [ ] Make margin functions for snp and nse
- [ ] See if curve fitting is needed with appropriate `penalty` based on DTE and strike
- [ ] Compute `computedmgn` and populate df_chains

In [ ]:
# imports
import asyncio
import numpy as np
from utils import get_pickle, get_mkt_prices, get_lots

## Get underlying prices

In [ ]:
# Get und_prices
unds = get_pickle(DATAPATH / 'unds.pkl')
df_unds = pd.DataFrame.from_dict(unds.items())
df_unds.columns = ['symbol', 'contract']

In [ ]:
TEMP_PATH = DATAPATH.parent / 'ztemp' / f"{MARKET.lower()}_df_und_prices.pkl"


In [ ]:
# # Get prices of unds.
# df_prices = asyncio.run(get_mkt_prices(port=PORT, contracts=df_unds.contract))
# from utils import pickle_me
# pickle_me(df_prices, TEMP_PATH)

In [ ]:
# ! for faster notebook loads
df_und_prices = get_pickle(TEMP_PATH)

df_und_prices.head()

## Make the `chains`, check and save

In [ ]:
import asyncio
from utils import make_chains, pickle_me

In [ ]:
TEMP_CHAIN = DATAPATH.parent / 'ztemp' / f"{MARKET.lower()}_df_chains.pkl"

In [ ]:
# # Make chains for underlying contracts
# contracts = df_unds.contract

# df_chains = asyncio.run(make_chains(port=PORT, contracts=contracts, MARKET=MARKET))
# pickle_me(df_chains, TEMP_CHAIN)

In [ ]:
# ! for faster noetbook loads
df_chains = get_pickle(TEMP_CHAIN)

In [ ]:
df_chains.head()

In [ ]:
# get lots
lots = get_pickle(DATAPATH / 'lots.pkl', print_msg=False)
if not lots:
    unds = get_pickle(DATAPATH / 'unds.pkl')
    lots = {s: get_lots(c) for s, c in unds.items()}


In [ ]:
# merge lots into df_und_prices
df_und_prices = df_und_prices.assign(lot = df_und_prices.symbol.map(lots),
                                     mgnCompute = np.nan)

## Get the closest strikes to underlying

In [ ]:
from utils import get_closest_values
import numpy as np

In [ ]:
def get_strike_closest_to_und(df_chains: pd.DataFrame, 
                              how_many: int= -1) -> pd.DataFrame:
    
    """
    Gets option contracts closest to strike for every expiry\n
    For SNP only the lowest dte is taken\n
    Useful to get reference margins.
    int: -1 for closest Put
    """

    if set(df_chains.exchange.to_numpy()).pop() == 'SNP':
        df_chains = df_chains.loc[df_chains.groupby(['symbol', 'right', 'strike']).dte.idxmin()]\
                            .reset_index(drop=True)

    strk_near_und = df_chains.groupby(['symbol', 'dte'])[['strike', 'undPrice']]\
        .apply(lambda x: get_closest_values(x.strike, 
                                            x.undPrice.min(), 
                                            how_many))
    strk_near_und.name = 'strk_near_und'

    df_ch1 = df_chains.set_index(['symbol', 'dte']).join(strk_near_und)
    df_ch2 = df_ch1[df_ch1.apply(lambda x: x.strike in x.strk_near_und, axis=1)] \
                            .reset_index()
    
    return df_ch2

In [ ]:
df_ch2 = get_strike_closest_to_und(df_chains)

In [ ]:
df_ch2.head()

## Find margins of the closest strikes
### ... and put it to chains

In [ ]:
# Find margins for options with strike closest to unds

from utils import make_a_raw_contract, get_margins
from ib_async import MarketOrder

# Integrate lots
df_ch3 = df_ch2.assign(lot=df_ch2.symbol.map(lots))


In [ ]:
df_ch3

In [ ]:
opt_contracts = [make_a_raw_contract(symbol=symbol, MARKET=MARKET, secType='OPT', strike=strike, right=right, expiry=expiry)
 for symbol, strike, right, expiry in zip(df_ch3.symbol, df_ch3.strike, df_ch3.right, df_ch3.expiry)]

orders = [MarketOrder(action='SELL', totalQuantity=qty) for qty in df_ch3.lot]

In [ ]:
df_margins = asyncio.run(get_margins(port=PORT, contracts=opt_contracts, orders=orders))

In [ ]:
df_margins[~df_margins.margin.isnull()]

## Integration of margins

In [ ]:
#Integration of margins

cols = ['symbol', 'expiry', 'strike', 'right']
mgn_cols = cols + ['margin', 'comm']
df_ch4 = pd.merge(df_ch3, df_margins[mgn_cols], on=cols, suffixes=[False, '_y']).drop(columns=['strk_near_und'])

chain_cols = ['symbol', 'expiry']
ch4_cols = chain_cols + ['lot', 'margin', 'comm']

df_out = pd.merge(df_chains, df_ch4[ch4_cols], on=chain_cols, suffixes=[False, '_y'])

In [ ]:
# fill NaNs with max_margins and comms

max_margins = df_out.groupby('symbol').margin.max().to_dict()
margin = df_out.margin.fillna(df_out.symbol.map(max_margins))


In [ ]:
max_comms = df_out.groupby('symbol').comm.max().to_dict()
comm = df_out.comm.fillna(df_out.symbol.map(max_comms))

In [ ]:
df_out1 = df_out.assign(margin = margin, comm = comm)

In [ ]:
df_out1[df_out1.margin.isnull()]

## For SNP

In [ ]:
# margin_compute is 30% of value
gt_16_67 = df_und_prices.price > 16.67

# margin_compute is 30% of $5 per share
gt_5_lt_16_67 = (df_und_prices.price > 5) & (df_und_prices.price <= 16.67)

# margin_compute is 100% of stock value
lt_5 = (df_und_prices.price > 2.5) & (df_und_prices.price <= 5)

# margin_compute is 100% of $2.5
lt_2_5 = (df_und_prices.price <= 2.5)

df_und_prices.loc[gt_16_67, 'mgnCompute'] = df_und_prices[gt_16_67].price * df_und_prices[gt_16_67].lot * 0.3


In [ ]:
condition = df_und_prices.mgnCompute.isnull()
condition = df_und_prices.symbol == 'CMG'
df_und_prices[condition]

In [ ]:
13000/1401/100

In [ ]:
get_pickle(DATAPATH / 'df_naked_targets.pkl')

In [ ]:
pickled_chains = DATAPATH / 'df_chains.pkl'
MINUTES_GAP = 15 # Ensures that the files are updated at least 4 times an hour!

# check age of `df_chains.pkl`
pickle_chains_age = get_file_age(pickled_chains)

# get the pickled chains and remove dte <=0
df_chains = get_pickle(pickled_chains)

td_in_minutes = pickle_chains_age.td.total_seconds()/60

if td_in_minutes > MINUTES_GAP:
    df_chains = update_chains_dte(df_chains, MARKET)
    df = compute_sdev_right(df_chains)
    pickle_me(df, pickled_chains)

### Calculate `sdev` and `right`

In [ ]:
from utils import get_a_stdev

In [ ]:
def compute_strike_sd_right(df: pd.DataFrame) -> pd.DataFrame:
    """Computes strike's standard deviation and right for option chains
    
    Note:
    ----
    Function needs `iv`, `undprice` and `dte` columns in df"""

    # remove dtes <= 0 to prevent math failure
    df = df[df.dte > 0].reset_index(drop=True)

    # compute strike's standard deviation
    df = df.assign(sigma=df[['iv', 'undPrice', 'dte']].\
                    apply(lambda x: get_a_stdev(x.iv, x.undPrice, x.dte), axis=1))

    df = df.assign(strike_sdev = (df.strike - df.undPrice) / df.sigma)

    # determine the right
    df = df.assign(right = df.strike_sdev.apply(lambda strike_sdev: 'P' if strike_sdev < 0 else 'C'))

    return df

In [ ]:
def target_options_with_adjusted_sdev(df_chains: pd.DataFrame,
                                      STDMULT: float,
                                      how_many: int,
                                      DTESTDEVLOW: float, 
                                      DTESTDEVHI: float,
                                      DECAYRATE: float,
                                      MARKET_IS_OPEN: bool) -> pd.DataFrame:
    
    """Adjust the standard deviation to DTE, penalizes DTES closer to zero"""

    # Get the extra SD adjusted to DTE
    # xtra_sd = 1-(df_chains.dte/100)

    MARKET = df_chains.exchange.unique()[0]

    _vars = Vars(MARKET)

    # Factor a bump to dev if market is not open
    if MARKET_IS_OPEN:
        GAPBUMP = 0
    else:
        GAPBUMP = _vars.GAPBUMP

    xtra_sd = df_chains.dte.apply(lambda dte: sdev_for_dte(dte=dte,
                                                           DTESTDEVLOW=DTESTDEVLOW, 
                                                           DTESTDEVHI=DTESTDEVHI,
                                                           DECAYRATE=DECAYRATE,
                                                           GAPBUMP=GAPBUMP
                                                           ))

    # Build the series for revised SD
    sd_revised = STDMULT + xtra_sd if STDMULT > 0 else STDMULT - xtra_sd

    # Identify the closest standerd devs to the revised SD\
    df_ch = df_chains.assign(sd_revised=sd_revised)
    closest_sdevs = df_ch.groupby(['symbol', 'dte'])[['sdev', 'sd_revised']]\
        .apply(lambda x: get_closest_values(x.sdev, 
                                            x.sd_revised.min(), 
                                            how_many))
    closest_sdevs.name = 'sdev1' 

    # Join the closest chains to the closest revised SD
    df_ch1 = df_ch.set_index(['symbol', 'dte']).join(closest_sdevs)

    # Get the target chains
    df_ch2 = df_ch1[df_ch1.apply(lambda x: x.sdev in x.sdev1, axis=1)] \
                        .reset_index()
    
    return df_ch2